In [42]:
import constraint as c

In [43]:
# two example problems from russell and norvig

australia = "SA:WA NT Q NSW V; NT:WA Q; NSW:Q V; T:"
    
# we use triple quotes since we have newlines 
usa = """WA:OR ID; OR:ID NV CA; CA:NV AZ; NV:ID UT AZ; ID:MT WY UT;
       UT:WY CO AZ; MT:ND SD WY; WY:SD NE CO; CO:NE KA OK NM; NM:OK TX;
       ND:MN SD; SD:MN IA NE; NE:IA MO KA; KA:MO OK; OK:MO AR TX;
       TX:AR LA; MN:WI IA; IA:WI IL MO; MO:IL KY TN AR; AR:MS TN LA;
       LA:MS; WI:MI IL; IL:IN; IN:KY; MS:TN AL; AL:TN GA FL; MI:OH;
       OH:PA WV KY; KY:WV VA TN; TN:VA NC GA; GA:NC SC FL;
       PA:NY NJ DE MD WV; WV:MD VA; VA:MD DC NC; NC:SC; NY:VT MA CA NJ;
       NJ:DE; DE:MD; MD:DC; VT:NH MA; MA:NH RI CT; CT:RI; ME:NH;
       HI:; AK:"""

In [44]:
def parse_map(neighbors):
    """Given a string like 'X:Y Z; Y:Z' returns a tuple of
    regions and adjoining pairs.  The syntax is a region
    name followed by ':' followed by 0 or more region names,
    followed by ';', repeated for each region.  Given 'X:Y'
    you don't need 'Y:X'.  Example:
      >>> parse_map('X:Y Z; Y:Z') 
      ([X,Y,Z], [(X,Y),(Y,X),(X,Z),(Z,X),(Y,Z),(Z,Y)])
    """
    adjoins = set()
    regions = set()
    specs = [spec.split(':') for spec in neighbors.split(';')]
    for (r1, r1_neighbors) in specs:
        # remove any whitespace from r1's ends
        r1 = r1.strip();
        # add it to the set of regions
        regions.add(r1)
        for r2 in r1_neighbors.split():
            # add r2 to the set of regions
            regions.add(r2)
            # add (r1, r2) to set of adjoining pairs
            adjoins.add((r1,r2))
    # return tuple with list of regions and
    # list of adjoining pairs
    return (list(regions), list(adjoins))

In [45]:
parse_map(australia)

(['WA', 'Q', 'T', 'V', 'SA', 'NT', 'NSW'],
 [('NT', 'Q'),
  ('SA', 'V'),
  ('NSW', 'Q'),
  ('SA', 'NT'),
  ('SA', 'NSW'),
  ('NSW', 'V'),
  ('SA', 'Q'),
  ('SA', 'WA'),
  ('NT', 'WA')])

In [46]:
def color (map, colors=['red','green','blue'], solver=c.BacktrackingSolver()):
    # create a new cconstraint problem
    p = c.Problem(solver)
    # parse_map returns a typle of regions and pairs
    (vars, pairs) = parse_map(map)
    # each region becomes a variable
    p.addVariables(vars, colors)
    # each pair of adjoining regions becomes a != constraint
    for (v1, v2) in pairs:
        p.addConstraint(lambda x,y: x!=y, [v1, v2])
    # getSolutionIter() returns a stream of solutions
    solution = p.getSolution()
    if solution:
        # print the variable and it's value,
        # i.e., the region name and its color
        for v in sorted(vars):
            print "%s:%s; " % (v, solution[v]),
        print
    else:
        print "No solution found :-("

In [47]:
color(australia, ['red','green','blue'])

NSW:green  NT:green  Q:red  SA:blue  T:blue  V:red  WA:red 


In [48]:
# four colors will be enuf
color(usa, ['red','green','blue','yellow'])

AK:yellow  AL:green  AR:green  AZ:red  CA:yellow  CO:yellow  CT:blue  DC:blue  DE:blue  FL:blue  GA:yellow  HI:yellow  IA:blue  ID:yellow  IL:green  IN:yellow  KA:red  KY:green  LA:blue  MA:yellow  MD:green  ME:yellow  MI:blue  MN:green  MO:yellow  MS:yellow  MT:green  NC:green  ND:blue  NE:green  NH:blue  NJ:green  NM:green  NV:blue  NY:blue  OH:red  OK:blue  OR:green  PA:yellow  RI:green  SC:blue  SD:yellow  TN:blue  TX:yellow  UT:green  VA:yellow  VT:green  WA:blue  WI:yellow  WV:blue  WY:blue 


In [49]:
color(usa, ['red','green','blue','yellow'], c.MinConflictsSolver())

AK:yellow  AL:red  AR:yellow  AZ:yellow  CA:blue  CO:yellow  CT:blue  DC:blue  DE:yellow  FL:blue  GA:green  HI:green  IA:blue  ID:green  IL:yellow  IN:red  KA:blue  KY:green  LA:blue  MA:yellow  MD:green  ME:green  MI:blue  MN:red  MO:red  MS:green  MT:blue  NC:yellow  ND:green  NE:green  NH:blue  NJ:green  NM:blue  NV:red  NY:red  OH:red  OK:green  OR:yellow  PA:blue  RI:red  SC:blue  SD:yellow  TN:blue  TX:red  UT:blue  VA:red  VT:green  WA:blue  WI:green  WV:yellow  WY:red 
